In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
import warnings
warnings.filterwarnings('ignore')

On se propose de procéder à une certaine selection des variables en utilisant des méthodes économétriques de significativité des variables.

L'idée : étudier un problème linéaire de la forme 
$y= \beta_0+ \beta_1x_1 + \beta_2x_2 + \beta_3x_3 + ... + \beta_{num features}x_{num features} + \epsilon$

Puis on test : H_0 : $\beta_i = 0$ pour tous i  

Cele peut nous donner des indications sur quelles variables utiliser 

#### Problème :

Notre target est binaire -> quelles implications ?

## Selection de Features par t-test

In [3]:
import numpy as np
import statsmodels.api as sm

## Data

In [80]:
from transplant.data.learningset import Learningset

learningset = Learningset()

X_train, X_test = learningset.get_data_merged_dynamic_flatten_full(full_df=True)
#X_train, X_test, y_train, y_test, X_col = learningset.get_data_merged_dynamic_flatten_full()

In [104]:
#X_train.dtypes

In [103]:
X_col= X_train.columns.drop('target')

J'ai eu des petits problèmes avec le format des noms ( ne supporte pas les espaces, les points et les parenthèses )

In [99]:
X_train=X_train.rename(columns = {'BIS SR_mean':'BIS_SR_mean','B.I.S_mean':'BIS_mean','SvO2 (m)_mean':'SvO2m_mean',
                                  'BIS SR_std':'BIS_SR_std','B.I.S_std':'BIS_std','SvO2 (m)_std':'SvO2m_std',
                                 'BIS SR_max':'BIS_SR_max','B.I.S_max':'BIS_max','SvO2 (m)_max':'SvO2m_max',
                                 'BIS SR_min':'BIS_SR_min','B.I.S_min':'BIS_min','SvO2 (m)_min':'SvO2m_min'})

## Feature selection for test

Ce n'est peut être pas une bonne pratique de mettre toutes les variables d'un coup mais on peut essayer

In [105]:
liste_full=X_col[0]
for i in X_col[1:] :
    liste_full+=' + ' + i

results = smf.ols('target ~ '+liste_full, data=X_train).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.601
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     1.361
Date:                Sat, 26 Jan 2019   Prob (F-statistic):             0.0474
Time:                        10:53:55   Log-Likelihood:                -62.036
No. Observations:                 241   AIC:                             378.1
Df Residuals:                     114   BIC:                             820.6
Df Model:                         126                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [95]:
#dir(results) # to see all attributes

#### Variables significatives au niveau 10 %

On pourrait essayer des modèles avec ça 

In [108]:
results.pvalues[results.pvalues<0.1]

start_operation_month    0.054758
ends_operation_month     0.054758
PAPsys_mean              0.045796
Temp_mean                0.023702
BIS_std                  0.061687
PAPdia_std               0.097504
PAPmoy_std               0.067849
PAPsys_std               0.049600
BIS_max                  0.021007
PAPmoy_max               0.039046
PAPsys_max               0.033081
SvO2m_max                0.066449
PAPmoy_min               0.095809
PAPsys_min               0.046720
dtype: float64

## Selections plus fines 

A poursuivre ...

In [109]:
liste_10_premiers_static=X_train.columns[0]
for i in X_train.columns[1:10] :
    liste_10_premiers_static+=' + ' + i
liste_10_premiers_static

'Age_donor + Aspirations_donor + BMI_donor + Donneur_CPT + Insuffisance_renale + LAS + PAPS + PFO + PF_donor + Poids'

In [110]:
results = smf.ols('target ~ '+liste_10_premiers_static, data=X_train).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.112
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     2.891
Date:                Sat, 26 Jan 2019   Prob (F-statistic):            0.00202
Time:                        10:56:34   Log-Likelihood:                -158.37
No. Observations:                 241   AIC:                             338.7
Df Residuals:                     230   BIC:                             377.1
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               1.0422    